<a href="https://colab.research.google.com/github/anujsahani01/EngMarBART/blob/main/Bidirectional(en_mr).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install datasets
# !pip install --upgrade accelerate
# !pip install wandb
# !pip install --no-cache-dir transformers sentencepiece
# !pip install evaluate
# !pip install rouge_score
# !pip install sacremoses
# !pip install fasttext
# !pip install sacrebleu

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:
from huggingface_hub import notebook_login
notebook_login()

In [35]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [5]:
from datasets import load_dataset
dataset = load_dataset('anujsahani01/processed_en_mr')

In [6]:
dataset

DatasetDict({
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 77723
    })
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 310890
    })
})

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="mr_IN", model_max_length = 512)
tokenizer.bos_token = tokenizer.eos_token

In [8]:
tokenizer

MBart50TokenizerFast(name_or_path='facebook/mbart-large-50', vocab_size=250054, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']}, clean_up_tokenization_spaces=True)

In [9]:
special_tokens_dict = {"eos_token": "<CLS>"}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("We have added", num_added_toks, "tokens")

We have added 1 tokens


In [10]:
out = tokenizer(dataset['train']['english'][0], text_target =  dataset['train']['marathi'][0],  max_length= 100, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')

In [11]:
len(out['input_ids'][0])

100

In [12]:
decoded_preds = tokenizer.batch_decode(out['input_ids'])
print(decoded_preds)

['en_XX Besides Aamir Khan, the film also stars Amitabh Bachchan, Katrina Kaif and Fatima Sana Shaikh in the lead roles Meanwhile, Operation Shakti was also launched in Lucknow to tackle crimes against women in the state On being informed about the incident, police reached the spot and seized the body PM then decided to undertake an aerial survey of flood affected areas of North Gujarat this afternoon, after the swearingin ce<CLS>']


In [13]:
tokenizer.batch_decode(out['labels'])

["mr_IN दिवाळीच्या मुहुर्तावर प्रेक्षकांच्या भेटीला येणाऱ्या या चित्रपटामध्ये आमिर खान आणि अमिताभ बच्चन यांच्यासह कतरिना कैफ, फातिमा सना शेख यांच्याही प्रमुख भूमिका आहेत. दुसरीकडे, उत्तर प्रदेशची राजधानी असलेल्या लखनौ शहरात महिलावर होणाऱ्या अत्याचाराविरोधात 'ऑपरेशन शक्ति' अभियान सुरू करण्यात आलं आहे. घटनेची माहिती मिळताच पोलीस घटनास्थळी दाखल झाले आणि त्यांची मुलीचं शव ताब्यात घेतलं. नवीन राष्ट्रपती<CLS>"]

In [14]:
import numpy as np

In [15]:
max_length = 85
context_length = 85

def tokenize(data):
  data = tokenizer(data['english'], text_target =  data['marathi'],  max_length= max_length, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')
  inp_ids = []
  if len(data['input_ids']) != 0:
    for length, ids in zip(data["length"], data["input_ids"]):
      inp_ids.append(ids)
    label = []
    for length, lb in zip(data["length"], data["labels"]):
      label.append(lb)


  return {"input_ids": inp_ids,
          'labels' : label}

In [16]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns =  dataset["train"].column_names,
)

In [17]:
tokenized_dataset

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 77723
    })
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 310890
    })
})

In [18]:
inp = tokenized_dataset['train']['input_ids']

In [19]:
arr = []
import pandas as pd
df_final = pd.DataFrame()
df_final['tokenized_eng'] = inp

In [20]:
df_final['length'] = df_final['tokenized_eng'].apply(
    lambda row: len(row)
)

In [21]:
print(df_final['length'].min())
print(df_final['length'].max())
print(df_final['length'].min())

85
85
85


In [22]:
! pip install -U neptune transformers

In [23]:
from getpass import getpass
from transformers.integrations import NeptuneCallback
import neptune

In [24]:
my_project = "anujsahani01/ITStudio"
run = neptune.init_run(
    project="anujsahani01/ITStudio",
    api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMzNmZWY5My0yOGY1LTRiZDItODA1MS05ZTM3MTc0MTI2NTYifQ==',
    source_files= ["/content/drive/MyDrive/Mbart(Finetuning).ipynb"]
)


neptune_callback = NeptuneCallback(run=run)
# run["train_dataset"].track_files("./train.csv")


<ipython-input-24-b6d931fdc011>:2: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/anujsahani01/ITStudio/e/MODEL-81


In [25]:
from transformers import AutoModel

from transformers import AutoModelForSeq2SeqLM, AutoConfig
model_checkpoint  = "facebook/mbart-large-50"
config = AutoConfig.from_pretrained(model_checkpoint, vocab_size=len(tokenizer), use_auth_token=True,)

config.vocab_size = tokenizer.vocab_size
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config = config, ignore_mismatched_sizes=True).to('cuda')
model.resize_token_embeddings(len(tokenizer))

Embedding(250055, 1024)

In [26]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [27]:
! pip install evaluate

In [28]:
import evaluate
metric = evaluate.load("sacrebleu")

In [29]:
import numpy as np


def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # In case the model returns more than the prediction logits
  if isinstance(preds, tuple):
      preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100s in the labels as we can't decode them
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}

In [30]:
# params = {'lr':0.0005, 'max_steps': 9000, 'warmup_steps': 30, 'weight_decay': 0.1, 'per_device_train_batch_size': 9, 'per_device_eval_batch_size':9, 'evaluation_strategy' :"no", 'save_total_limit' :1, 'num_train_epochs': 1, 'predict_with_generate' : True, 'fp16' : False, 'push_to_hub' : True, 'remove_unused_columns' :False}
params = {'lr':0.0005, 'max_steps': 9000, 'warmup_steps': 30, 'weight_decay': 0.01, 'per_device_train_batch_size': 11, 'per_device_eval_batch_size': 11, 'evaluation_strategy' :"no", 'save_total_limit' :1, 'num_train_epochs': 1, 'predict_with_generate' : True, 'fp16' : False, 'push_to_hub' : True, 'remove_unused_columns' :False, 'dropout': 0.5}

In [31]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    f"./finetuned_en2mr",
    evaluation_strategy= params['evaluation_strategy'],
    # eval_steps=1500,
    save_strategy="epoch",
    learning_rate= params['lr'] ,
    per_device_train_batch_size=params['per_device_train_batch_size'],
    per_device_eval_batch_size= params['per_device_eval_batch_size'],
    warmup_steps = params['warmup_steps'],
    weight_decay= params['weight_decay'],
    save_total_limit= params['save_total_limit'],
    num_train_epochs= params['num_train_epochs'],
    predict_with_generate= params['predict_with_generate'],
    fp16= params['fp16'],
    push_to_hub= params['push_to_hub'],
    max_steps = params['max_steps'],
    remove_unused_columns= params['remove_unused_columns'],
    report_to="neptune"
    # predict_with_generate = True
)

In [32]:
neptune_callback = NeptuneCallback(run=run)

In [36]:
trainer = Seq2SeqTrainer(
    model.to('cuda'),
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)
    # callbacks=[neptune_callback])

/content/./finetuned_en2mr2 is already a clone of https://huggingface.co/anujsahani01/finetuned_en2mr2. Make sure you pull the latest changes with `repo.git_pull()`.


In [37]:
for cb in trainer.callback_handler.callbacks:
  if isinstance(cb, NeptuneCallback):
    trainer.callback_handler.remove_callback(cb)

In [38]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,5.995300
1000,4.362500
1500,4.098800
2000,3.927600
2500,3.812000
3000,3.730500
3500,3.646900
4000,3.573900
4500,3.504600
5000,3.459400


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


TrainOutput(global_step=9000, training_loss=3.675078043619792, metrics={'train_runtime': 2823.7876, 'train_samples_per_second': 35.059, 'train_steps_per_second': 3.187, 'total_flos': 1.780898144256e+16, 'train_loss': 3.675078043619792, 'epoch': 0.32})

In [39]:
metrics = trainer.state.log_history

In [40]:
for key, value in params.items():
  run[key] = value

In [41]:
lr = []
train_loss = []
steps =  []
epoch = []

In [42]:
for i in range(len(metrics)-1):
  train_loss.append(metrics[i]['loss'])
  steps.append(metrics[i]['step'])
  lr.append(metrics[i]['learning_rate'])
  epoch.append(metrics[i]['epoch'])

In [43]:
for i in range(len(train_loss)):
  run['train/loss'].append(train_loss[i])

for i in range(len(epoch)):
  run['epoch'].append(epoch[i])

for i in range(len(lr)):
  run["learning_rate"].append(lr[i])

for i in range(len(steps)):
  run["steps"].append(steps[i])

In [ ]:
trainer.push_to_hub()

In [ ]:
! pip install sacremoses

In [ ]:
from transformers import AutoModelForCausalLM, TranslationPipeline, Text2TextGenerationPipeline, pipeline

model_ckpt = "anujsahani01/finetuned_en2mr"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
pipe = pipeline('translation_en_to_mr', model_ckpt)

In [ ]:
src_text = 'he is playing outside'
pred = pipe(src_text, num_return_sequences=3)
pred

In [ ]:
run["train_dataset"].track_files("./train.csv")

In [ ]:
from evaluate import TranslationEvaluator
metric = TranslationEvaluator(task = 'translation', default_metric_name = "bleu")
evaluation_results = metric.compute(model_or_pipeline = model, tokenizer = tokenizer,  metric = 'bleu', data = dataset['test'][:100] , device = 0, split = 'test' ,input_column = 'english', label_column = 'marathi')

In [ ]:
run["bleu"].append(evaluation_results["bleu"])

In [44]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/anujsahani01/ITStudio/e/MODEL-81/metadata
